In [1]:
from unityagents import UnityEnvironment
import numpy as np
import agent
import random

## Preparing the environment for training

In [2]:
path_to_env='../Reacher_Linux/Reacher.x86_64'

env = UnityEnvironment(file_name=path_to_env)

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)
# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:\n', states[0])

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like:
 [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


In [ ]:
env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)

# Tests for memory usage
#myAgent = agent.Agent(3)
#l = myAgent.memory_size

for i in range(20):
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actionsClipped = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    print(actionsClipped)
    env_info = env.step(actionsClipped)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    
    myAgent.add(states[0], actions[0], rewards[0], next_states[0], dones[0])
    for N, j in enumerate(rB.episode_len):
        l_now = len(myAgent.individual_memory)
        if ( l_now >= j):
            past_states = [myAgent.individual_memory[k].state for k in range(l_now-j,l_now)]
            past_actions = [myAgent.individual_memory[k].action for k in range(l_now-j,l_now)]
            past_rewards = [myAgent.individual_memory[k].reward for k in range(l_now-j,l_now)]
            past_next_states = [myAgent.individual_memory[k].next_state for k in range(l_now-j,l_now)]
            past_dones = [myAgent.individual_memory[k].done for k in range(l_now-j,l_now)]
            rB.add(past_states, past_actions, past_rewards, past_next_states, past_dones, N)
    
    states = next_states                               # roll over states to next time step
    #if np.any(dones):                                  # exit loop if episode finished
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [3]:
num_critics = 1 # Define the number of independant critics wanted

env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)

# Penalty coefficient for high actions
alphaTor = 0.005


# Create Critic Agents
critics = [] # agent list
crit_lrs = 1e-4 # np.exp(-2* np.log(10) * np.random.rand(20)) * 10**(-2)
for i in range(num_critics):
    model = agent.CriticModel(state_size, action_size)
    critic = agent.CriticAgent(model, crit_lrs)
    critics.append(critic)

# Create Actor Agents
workers = [] # agent list
crit2work = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
# work_lrs = [1e-4,1e-3,1e-5,2e-4,2e-3,1e-4,2e-5,3e-4,3e-5,3e-4,2e-5,1e-3,1e-4,1e-5,2e-4,3e-4,3e-3,2e-5,5e-5,5e-4]
for i in range(num_agents):
    model = agent.ActorModel(state_size, action_size)
    critic = critics[0] #crit2work[i]]
    actor = agent.ActorAgent(model, critic, lr = 1e-4,memory_size = 3)
    actor.noise.theta = 0.1 # set theta in OUnoise
    actor.noise.sigma = 0.1 # set sigma in OUnoise
    workers.append(actor)
    

# Create the shared replay buffer
shared_memory = agent.ReplayBuffer(action_size, buffer_size = 1e5, batch_size =512, episode_len=[1])
scoreList = []
count = 0
while True:
    scores = np.zeros(num_agents)
    while True:
        # Select an action for each agent
        actions = []
        for i, worker in enumerate(workers):
            state = states[i]
            action = worker.act(state)
            actions.append(action)

        # send the actions to the environment
        env_info = env.step(actions)[brain_name]

        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)

        # Memorize last actions
        for i, worker in enumerate(workers):
            # add a penalty for excessive torque (high action values)
            myReward = rewards[i] - alphaTor * np.abs(actions[i].max())
            worker.add(states[i], actions[i], myReward, next_states[i], dones[i])
        # send individual memory to shared memory
        for spot, spotLength in enumerate(shared_memory.episode_len):
            for worker in workers:
                l = len(worker.individual_memory)
                if ( l >= spotLength): # check if the agent has enough steps in memory for the current shared memory spot
                    currentRange = range(l-spotLength, l)
                    past_states = [worker.individual_memory[k].state for k in currentRange]
                    past_actions = [worker.individual_memory[k].action for k in currentRange]
                    past_rewards = [worker.individual_memory[k].reward for k in currentRange]
                    past_next_states = [worker.individual_memory[k].next_state for k in currentRange]
                    past_dones = [worker.individual_memory[k].done for k in currentRange]
                    shared_memory.add(past_states, past_actions, past_rewards, past_next_states, past_dones, spot)
        # Train the critics
        # 1) generate a sample with the various length replaybuffers (use ReplayBuffer.sample and CriticAgent.evaluate)
        for i in range(len(shared_memory.episode_len)): # loop over the different spots
            if (len(shared_memory) > shared_memory.batch_size):
                Expstates, Expactions, Exprewards, Expnext_states, Expdones = shared_memory.sample(i) # sample the ith memory spot
            # 2) critic.learn
                for i, critic in enumerate(critics): # adapt for long episodes
                    # Expnext_state = Expnext_states[-1]
                    Expnext_actions = workers[i].act(Expnext_states)
                    Expstate, Expaction, Expestimated_reward = critic.evaluate(Expstates.squeeze(), Expactions.squeeze(), Exprewards, Expnext_states.squeeze(), Expnext_actions.squeeze(), Expdones.squeeze())
                    critic.learn(Expstate, Expaction, Expestimated_reward)
                #train the actors
                for worker in workers:
                    worker.learn(Expstate)
            states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    if True: # count%10 == 0:
        print('Total score (averaged over agents) for episode {}: {}'.format(count ,np.mean(scores)))
    scoreList.append([scores])
    count += 1
    if np.mean(np.array(scoreList)[-100:]) > 30:
        break
print("Total mean score: {}".format(np.mean(scoreList)))

Total score (averaged over agents) for episode 0: 0.30549999317154286
Total score (averaged over agents) for episode 1: 0.0549999987706542
Total score (averaged over agents) for episode 2: 0.3674999917857349
Total score (averaged over agents) for episode 3: 0.040499999094754456


KeyboardInterrupt: 

In [ ]:
scores

In [ ]:
Expstates, Expactions, Exprewards, Expnext_states, Expdones = shared_memory.sample(0)
Expnext_actions = workers[i].act(Expnext_states)
Expstate, Expaction, Expestimated_reward = critic.evaluate(Expstates.squeeze(), Expactions.squeeze(), Exprewards, Expnext_states.squeeze(), Expnext_actions.squeeze(), Expdones.squeeze())
#Expnext_states.squeeze().shape
Expestimated_reward.shape

In [ ]:
import torch
critic.target(torch.from_numpy(Expnext_states.squeeze()).float().to("cuda"), torch.from_numpy(Expnext_actions.squeeze()).float().to("cuda")).cpu().data.numpy()

In [ ]:
scoreList

In [ ]:
from matplotlib import pyplot as plt
for i in range(3):
    plt.plot( np.array(scoreList)[:,:,i], label=i)
    plt.legend()
    
plt.show()

In [ ]:
shared_memory.memory[0][-5]

In [ ]:
workers[0].noise.theta

In [ ]:
scoreList

In [ ]:
Expaction